# BMW Flask API installation:

update bashrc for qq user:

In [ ]:
#run once:

cleantext="
export HISTTIMEFORMAT="[%Y-%m-%d %H:%M:%S] "
HISTSIZE='INFINITY'; HISTFILESIZE='ANDBEYOND'

PS1='\e[37m\D{%H:%M}\e[91m[\e[90m\u@\h \e[33m\w\e[31m]\e[92m\n\$'

alias ll='ls -alF'
alias la='ls -A'
alias l='ls -CmF'
alias lr='ls -ltrh'
alias ufind="find / -name $1 2>/dev/null"

export PATH=$PATH:/home/${USER}/scripts  # currently not used
"

echo "$cleantext" >> /home/${USER}/.bashrc.sh

In [ ]:
# Check OS vesion:
cat /etc/os-release

(below commands are for RHEL distro)

In [ ]:
# install prereqs with elevated user:

# prod:
yum install -y libXcomposite libXcursor libXi libXtst libXrandr alsa-lib mesa-libEGL libXdamage mesa-libGL libXScrnSaver

# dev:
subscription-manager repos --enable=codeready-builder-for-rhel-8-x86_64-rpms # required for x11
yum install -y libXcomposite libXcursor libXi libXtst libXrandr alsa-lib mesa-libEGL libXdamage mesa-libGL libXScrnSaver xorg-x11-apps xorg-x11-xauth firefox

[Within VDI]
> download <mark>Anaconda3-2020.02-Linux-x86_64.sh</mark> from https://repo.anaconda.com/archive/ 

> download & install WinSCP from http://wuss.bmwgroup.net

> upload Anaconda3-2020.02-Linux-x86_64.sh to <mark>Telegraf host</mark> to /tmp or /home/qqky020/UI/install_files (but first create the folder for that - see below)


[Telegraf host]

In [ ]:
# create folders:
mkdir -p  /home/${USER}/UI/Flask ~/UI/install_files
# give execute rights to the installed:
chmod +x Anaconda3-2020.02-Linux-x86_64.sh
# Long press Enter; then "yes"; then "yes" (again)
# when completed - reload shell:
. ~/.bashrc
# install requirements for Flask API (maybe you need to update the path to requirements folder: check "flask_wapi_UAT")
cd  ~/UI/flask_wapi_UAT/requirements; `for': for file in $(ls) ; do pip install ./${file}; done  
    # check if all are required (wheel; tar.gz.. some might be duplicates)
. ../.flask run
# dev/test instance
flask run --host 0.0.0.0  
# prod
IP="$(hostname -I | awk '{print $1}')"
nohup flask run --host $IP &
# to close it type "fg" and press ctrl+c; #or "kill %1" #but be sure that is the only background process that is running!

[VDI]
> open preferred web browser

> open the UI: http://<mark>XX.X.XX.XX</mark>:8000 #replace with correct IP address <mark># note that the firewall port 8000 has to be opened!</mark>

[Telegraf host]

Backend & <b>cronjobs</b>: <mark>monitoring_services.sh</mark>

In [ ]:
# create backend script:
vi monitoring_services.sh
# paste the below (to be updated) #current version = v1.05; 2022.08.05 (Author: Michal Márkus)
########################################################################################

#!/bin/bash
#-x
# monitoring_services.sh


DATE=`date +'%m/%d/%Y %H:%M:%S'`
err_msg="not running @"

flask_path=/home/qqky020/UI/flask_wapi_UAT
cd $flask_path


services_check()
{

# Check status if services are running:


# GRAFANA:
        grafana_status()
        {
                grafana_check="$(curl -sL -I itahdnasrep.bmwgroup.net:3000/ping/api/health | grep HTTP | grep 200 | awk '{print $2}')";  # echo "$grafana_check"
                grafana_latency=$(curl -s -w 'Establish Connection: %{time_connect}s\nTTFB: %{time_starttransfer}s\nTotal: %{time_total}s\n' itahdnasrep.bmwgroup.net:3000/ping/api/health | egrep "Total: [1-9]");  # echo $grafana_latency  # test with 0
                [ $(eval echo \$"${service}_check") == 200 ] && [ -z "$latency" ] || echo -e "\n$DATE\n$grafana_check\n\n###" >> ${service}_high_latency.log && echo "${service} OK" >> ${service}_uptime.log || echo "${service}" $err_msg $DATE >> ${service}_uptime.log
        }


# HARVEST:

# QQ USER NEEDS TO BE ADDED TO REMOTE HOST & set up PWLESS SSH (or info needs to be posted to this host via Ansible...)


        harvest_status()
        {
                harvest_check="$(ssh -tt michal@itahdnasuathar.bmwgroup.net 'systemctl status harvest')"  # to be replaced with qq user!
                if [ "$(eval echo \$${service}_check) | sort -u | grep -v running | wc -l)" -gt 0 ]; then echo "${service}" $err_msg $DATE >> ${service}_uptime.log; else echo "${service} OK";fi
        }


# INFLUX:
        influx_status()
        {
                influx_check="$(curl -sL -I itahdnasrep.bmwgroup.net:8086 | grep HTTP | grep 200 | awk '{print $2}')"
                influx_latency=$(curl -s -w 'Establish Connection: %{time_connect}s\nTTFB: %{time_starttransfer}s\nTotal: %{time_total}s\n' itahdnasrep.bmwgroup.net:8086 | egrep "Total: [3-9]")  # test with 0
                [ $(eval echo \$"${service}_check") == 200 ] && [ -z "$latency" ] || echo -e "\n$DATE\n${service}_check\n\n###" >> ${service}_high_latency.log && echo "${service} OK" >> ${service}_uptime.log || echo "${service}" $err_msg $DATE >> ${service}_uptime.log
        }


###

#declare {nodered,ansible}="echo not configured yet "  # CURRENTLY NOT SET

###


# TELEGRAF:

        telegraf_status()
        {
                telegraf_check="$(systemctl | grep telegraf | sort -u | grep -v running | wc -l)"
                if ! [ "$(echo $telegraf_check)" == 0 ]; then echo "${service}" $err_msg$DATE >> ${service}_uptime.log; else echo "${service} OK" >> ${service}_uptime.log; fi
        }


# LOOP OVER SERVICES:

#for service in grafana harvest influx telegraf ansible nodered
for service in grafana influx telegraf #harvest #ansible nodered
    do
        ${service}_status
    done
}



ticket()
{

# Send info to NodeRed when service is down:
EPOCHNOW=`date -d "${DATE}" +"%s"`

    
generate_json()    
    c=0
    for((i=1;i<=5;++i))
        do
            dat=$(tac ${service}_uptime.log | sed -n "${i},1p" | cut -d@ -f2) 

            case $dat in
                ''|*OK) return 1 ;;
                *)

                epoch_dat=`date -d "${dat}" +"%s"`

                if [ "$(echo $EPOCHNOW-$epoch_dat|bc)" -le "360"  ] # less or equal to 360 seconds AKA 6 min (5min +1min grace time due to latency)
                    then c=$((c+1))
    #                if [ "$c" == 1 ]; then echo ${service}_down_since "$epoch_dat"; fi
                fi
                                    ;;
            esac
        done

    if [[ $c -ge 5 ]]; then echo "PLACEHOLDER for *ticket file with paramenters edited by sed will be passed to NodeRed*" | tee  ${service}_monitoring_ticket_`date +\%Y\%m\%d\%H\%M\%S`.json; else echo OK; fi

    # static IDs; variables to NodeRed via json: https://atc.bmwgroup.net/confluence/download/attachments/2076532016/InterfaceContract_EventMgmt_NAS_final.pdf?version=2&modificationDate=1646741380958&api=v2

    
for service in grafana influx telegraf #harvest #ansible nodered
    do
        generate_json(${service})
    done    
    
    
}


manage_logs()
{

# PAST INCIDENTS:
    # either use pwless ssh; or copy files to telegraf via Ansible playbook

#rm incidents_*.csv 2>/dev/null

for T in DAYS WEEKS MONTH;
    do declare t=${T,,}; #echo $t;
        RANGE=$(date -d "$date -1 ${t}" +"%s");

        #cat telegraf_uptime.log | while read line;
        ls *_uptime.log | xargs cat | sort -u | while read line;

        do

            x=$(echo $line |grep -v OK |cut -d@ -f2)
            # if x is number number then convert to epoch format (y):
            #if [[ $x == ?(-)+([0-9])  ]]; then  # x is NOT a number because "/" characters in time format!
            if ! [[ $x == ''  ]]; then
                y=$(date -d "$x" +"%s")  # && echo $y-$RANGE|bc;  # check difference
                if [ "$RANGE" -le "$y" ]; then  echo $line >> incidents_${t}.csv; fi
            fi
        done

    done

# add when it was restarted started//since when it is running, and how long it runs; was down
# can calculate uptime from last difference

mv incidents_days.csv today.csv 2>/dev/null
mv incidents_weeks.csv weekly.csv 2>/dev/null
mv incidents_month.csv montly.csv 2>/dev/null

}


services_check
ticket
manage_logs  # Past incidents tab; logrotation to be added


In [ ]:
# Edit the crontab via:
crontab -e
# paste the following (this script runs every minute):
*/1 * * * * source ~/.bashrc; /home/qq_XX/monitoring_services.sh

## 2022.08.05: STATUS for P1: `70% #progress`

### P1 (pending items):

> Check/debug: <b>monitoring_services.sh</b> `[3%]`: generate_report() --json part; <b>UI</b>: infra.html `[2%]` #effort

> 'Past Incidents' tab: add uptime `[5%]`

> Enable passwordless ssh for qq user; or copy required files ('systemctl status harvest' output; maintenance.csv [resource,service,start,end] from NodeRed to telegraf host) `[5%]`

> Maintenance mode `[10%]`

#### P2 plan:
- Ansible jobs 
- Performance metrics/alerts/jobs...
- Add http<mark>s</mark> & cert to flask

#### P3 plan:
- CSS formatting
- Check redundant packages at <mark>requirements</mark> & check which modules are being used, so that on prod we don't have to install anaconda (minimalistic approach)
- Configure <mark>X11</mark> for jupyter on telegraf host

Decommission:

rm -rf ~/anaconda3  # also remove anaconda stuff from ~/.bashrc

# Tool configurations

# Telegraf

### Info from Florian:
---
```
I prepared the Grid configuration already, so that the Telegraf can start to collect the data. 
Please take care that it can take up to 15 minutes with the initial data collection till it get reflected into the InfluxDB with the “prometheus” _measurements. 

StorageGRID requires a certificate authentication, so in addition I attached you the required certificates. 
Move them in the /etc/telegraf directory or subdirectory (modify tls_ca/tls_ca_cert & tls_key path in this case).

There are 3 configuration parts to be modified / checked. 

#1  modify the common Telegraf config (at the beginning of the config file)
[agent]
   interval = “60s”
   metric_batch_size = 5000
   metric_buffer_limit = 75000


#2  add the Storagegrid Input config
 [[inputs.prometheus]] 
   urls = ['https://10.2.62.68:9091/federate?match%5B%5D=%7Bjob%3D~%22.%2B%22%7D']
   metric_version = 2
   tls_ca = "/etc/telegraf/cacert.pem"
   tls_cert = "/etc/telegraf/cert.pem"
   tls_key = "/etc/telegraf/key.pem"
   insecure_skip_verify = true
   response_timeout = "59s"


#3 check your [outputs.influxdb_v2]] configuration. 
Telegraf will write the data into the according bucket you set here. 


After this restart the Telegraf (via cmd # sudo systemctl stop telegraf & # sudo systemctl start telegraf). 
15 Minutes after this, the InfluxDB will reflect the StorageGRID data.
``` 

> Source: mail @Fri 22/05/13 13:11


## UAT Telegraf config steps
Telegraf configuration
Telegraf agents located on: <b>ITAHDNASUATTEL</b>
There is 5 instance running
- SNMP trap receiver
- SNMP query for Cisco Switches
- VM server data receiver
installation folder:
`/etc/telegraf`
each telegraf has its own service
#### sytemctl status telegraf_broadcom.service
`/usr/bin/telegraf -config /etc/telegraf/telegraf_broadcom.conf -config-directory /etc/telegraf/telegraf_broadcom`
#### sytemctl status telegraf_cisco.service
`/usr/bin/telegraf -config /etc/telegraf/telegraf_cisco.conf -config-directory /etc/telegraf/telegraf_cisco`
#### sytemctl status telegraf_storage.service 
`/usr/bin/telegraf -config /etc/telegraf/telegraf_storage.conf -config-directory /etc/telegraf/telegraf_storage`
#### sytemctl status telegraf_traps.service
`/usr/bin/telegraf -config /etc/telegraf/telegraf_traps.conf -config-directory /etc/telegraf/telegraf_traps`
#### sytemctl status telegraf_system.service
`/usr/bin/telegraf -config /etc/telegraf/telegraf_system.conf -config-directory /etc/telegraf/telegraf_system`

To receive SNMP traps from AIQ UM two MIB file required to copied to the configured path where the MIB's name are important
- NETAPP.MIB
- OCUM.MIB (this is a renamed aiqum_9.9.mib)

# Influx

## Install Influx CLI and Modify <mark>bucket's retention</mark>
Install Influx CLI/Modify bucket's retention:
Download package from the following URL: https://docs.influxdata.com/influxdb/cloud/tools/influx-cli/?t=Windows
Install CLI to VDI: Because we haven't permission on the 'C:\Program Files' folder, need modify the original command:
Ori: Expand-Archive .\influxdb2-client-2.3.0-windows-amd64.zip -DestinationPath 'C:\Program Files\InfluxData' mv 'C:\Program Files\InfluxData\influxdb2-client-2.3.0-windows-amd64' 'C:\Program Files\InfluxData\influx'
Modified: Expand-Archive .\influxdb2-client-2.3.0-windows-amd64.zip -DestinationPath 'C:\InfluxData' mv 'C:\InfluxData\influxdb2-client-2.3.0-windows-amd64' 'C:\InfluxData\influx'
Use Powershell for the following
Before issuing the above command, navigate to the folder where you downloaded the CLI package. For example:
```
cd C:\Users"USERNAME"\Downloads`
mkdir C:\InfluxData`
Expand-Archive .\influxdb2-client-2.3.0-windows-amd64.zip -DestinationPath 'C:\InfluxData'
mv 'C:\InfluxData\influxdb2-client-2.3.0-windows-amd64' 'C:\InfluxData\influx'
Navigate to the C:\InfluxData\influx // because we cannot modify the 'path' variable, need to go to the folder where the influx.exe exists
Create an influx CLI's config for the remote host: .\influx config create -a -n CONFIGNAME -u URL -t TOKEN_WHICH_HAS_PROPER_PRIVILEGES -o ORGANIZATION
List bucket's current settings:
PS C:\InfluxData\influx> .\influx.exe bucket list ID Name Retention Shard group duration Organization ID Schema Type 834ba3f797c35789 BroadcomBES 1440h0m0s 24h0m0s f24c8a8d0e5f36e8 implicit f827bf73e326118b CiscoBackend 1440h0m0s 24h0m0s f24c8a8d0e5f36e8 implicit
Modify bucket's retention: Command reference: https://docs.influxdata.com/influxdb/v2.2/organizations/buckets/update-bucket/
.\influx bucket update -i BUCKET_ID -r NEW_RETENTION_TIME
```
Done
